# Model Context Protocol

An example of how to setup and call a basic MCP server using FastMCP.

You can find the code for the demo server in `server.py`

**Start the server**:

- Uvicorn
  ```bash
  uvicorn notebooks.model-context-protocol.fastmcp_server:mcp_server --reload
  ```
- FastMCP (and UV)
  ```bash
  uv run fastmcp run --transport streamable-http fastmcp_server.py
  ```


**Launch inspector**
```bash
npx @modelcontextprotocol/inspector
```


**Server URL**:

```text
    http://127.0.0.1:8000/mcp
```

**Docs**: 
- [FastMCP repo](https://github.com/jlowin/fastmcp)
- [FastMCP wiki](https://fastmcp.wiki/en/getting-started/welcome)
- [Authentication/Protected servers](https://github.com/jlowin/fastmcp?tab=readme-ov-file#authentication)

---
## Client setup

The server can be called using a http client:

```python
    client = Client(server_url)
```


In [ ]:
import json
import pandas as pd
from fastmcp import Client

# Connect the client to the server
server_url = "http://127.0.0.1:8000/mcp"
client = Client(server_url)


---
## Tools

Tools can be used to call functions on the server.

**List available tools**: 

```python
    client.list_tools()
```

**Call a tool**: 
```python
    client.call_tool("my_fancy_tool", **kwargs)
```


In [ ]:
# List available tools
async with client:
    tools = await client.list_tools()

for tool in tools:
    print(f"Tool: {tool.name}")
    print(f"- Description: {tool.description}")
    print(f"- Arguments: {tool.inputSchema['properties']}")
    print(f"- Returns: {tool.outputSchema['properties']['result']}\n")


In [ ]:
# Call a tool
a = 5
b = 6

async with client:
    result = await client.call_tool("add", {"a": a, "b": b})

for msg in result.content:
    print(f"add({a=}, {b=}) -> {msg.text}")


---
## Resources

Resources typically contain things like data and metadata.

**List available resources**: 

```python
    resources = client.list_resources()
```

**Read a resource**: 

```python
    client.read_resource(uri)
```


In [ ]:
# List available resources
async with client:
    resources = await client.list_resources()

for resource in resources:
    print(f"Resource: {resource.name}")
    print(f"- Description: {resource.description}")
    print(f"- URI: {resource.uri}\n")


In [ ]:
# Read a resource
uri = "data://some_table"
async with client:
    result = await client.read_resource(uri)

for resource in result:
    data = json.loads(resource.text)
    df = pd.DataFrame(data)
    display(df)


---
## Prompts

MCP servers can also be used to store prompts.

**List available prompts**:

```python
    client.list_prompts()
```

**Get a prompt**:

```python
    client.get_prompt("my_fancy_prompt", **kwargs)
````


In [ ]:
# List available prompts
async with client:
    prompts = await client.list_prompts()

for prompt in prompts:
    print(f"Prompt: {prompt.name}")
    print(f"- Description: {prompt.description}")
    print(f"- Arguments: {[arg.name for arg in prompt.arguments]}\n")


In [ ]:
extra_instructions = "Svara alltid på svenska."
async with client:
    result = await client.get_prompt("system_prompt", {"extra_instructions": extra_instructions})

for msg in result.messages:
    print(msg.content.text)
